# Loading the libraries

In [7]:
import os
jdbcPathUbuntu = '/usr/share/java/postgresql-42.2.23.jar'
jdbcPathMacOS = '/Users/cristianomartinsm/postgresql-42.2.23.jar'
jdbcPath = jdbcPathUbuntu
os.environ['PYSPARK_SUBMIT_ARGS'] = '--driver-class-path ' + jdbcPath + ' --jars ' + jdbcPath + ' pyspark-shell'
import databricks.koalas as ks
import numpy as np
import pandas as pd
import networkx as nx
from heapq import heapify, heapreplace#, heappop, heappush

# Splitting the lengthy edges

In [8]:
params = {'user': 'cristiano', 'password': 'cristiano'}

def loadMultiGraph():
	kdf = ks.read_sql_query('''	select	EDGE.IDVERTEXORIG_FK,
										EDGE.IDVERTEXDEST_FK,
										EDGE.IDEDGE,
										EDGE.LENGTH,
										EDGE.ONEWAY
								from	STREETSEGMENT as EDGE
								where	EDGE.UTILITYVALUE <> 0 ''',
					'jdbc:postgresql:afterqualifying', options=params)

	G = nx.MultiGraph()
	for row in kdf.itertuples():
		dictRow = row._asdict()

		G.add_edge(dictRow['idvertexdest_fk'], dictRow['idvertexorig_fk'], key=dictRow['idedge'], idedge=dictRow['idedge'], length=dictRow['length'])

	#G = nx.from_pandas_edgelist(kdf.to_pandas(), 'idvertexorig_fk', 'idvertexdest_fk', ['idedge', 'length'], create_using=nx.MultiGraph(), edge_key='idedge')
	
	return G

G = loadMultiGraph()
print(G.number_of_nodes())

240874


In [9]:
def reBuildGraph(G, edgesHeap, firstSplit):
    for item in edgesHeap:
        (heapValue, u, v, idedge, lengthOriginal, numSplit) = item
        #The number of segments the edge must be split into is 1 less the value stored in the heap
        numSplit = numSplit - 1
        if numSplit >= firstSplit:
            lengthSplitted = lengthOriginal/numSplit
            vertexStart = u

            #print(G[u][v][idedge]['length'], nx.dijkstra_path_length(G, u, v, weight='length'), numSplit, lengthSplitted, lengthOriginal)
            oldDistance = nx.dijkstra_path_length(G, u, v, weight='length')

            G.remove_edge(u, v, key=int(idedge))
            for i in range(numSplit - 1):
                vertexEnd = str(idedge) + '_' + str(i + 1)
                G.add_edge(vertexStart, vertexEnd, key=vertexEnd, idedge=vertexEnd, length=lengthSplitted)
                vertexStart = vertexEnd
            keyLast = str(idedge) + '_' + str(numSplit)
            G.add_edge(vertexStart, v, key=keyLast, idedge=keyLast, length=lengthSplitted)

            #print(nx.dijkstra_path_length(G, u, v, weight='length'))
            newDistance = nx.dijkstra_path_length(G, u, v, weight='length')
            if round(oldDistance, 7) != round(newDistance, 7):
                print("ERROR IN DISTANCES:", oldDistance, newDistance)

    return G

In [10]:
def splitEdges(precision=1, maxDistance=None):
    G = loadMultiGraph()

    firstSplit = 2
    #The value must be negative because the data structure is a min heap
    edgesHeap = [(-1*data['length'], u, v, data['idedge'], data['length'], firstSplit) for u, v, data in G.edges(data=True)]
    heapify(edgesHeap)
    
    newLength = G.number_of_edges()
    if precision != 0:
        for i in range(len(edgesHeap) * precision):
            #The value must be multiplied by -1 because the data structure is a min heap
            if maxDistance != None and -1 * edgesHeap[0][0] <= maxDistance:
                break
            
            #(heapValue, u, v, idedge, lengthOriginal, numSplit) = heappop(edgesHeap)
            (heapValue, u, v, idedge, lengthOriginal, numSplit) = edgesHeap[0]

            #The value must be negative because the data structure is a min heap
            heapValue = -1 * lengthOriginal/numSplit

            #The numSplit is prepared for the next time the edge may be splitted (numsplit + 1)
            heapreplace(edgesHeap, (heapValue, u, v, idedge, lengthOriginal, numSplit + 1))

            newLength += 1

        #reBuildGraph(G, edgesHeap, firstSplit)

    lengths = sorted([-1 * item[0] for item in edgesHeap])

    return lengths, newLength

data, newLength = splitEdges(0)
data = np.asarray(data)
print(0, newLength)
data = data.reshape((data.shape[0], 1))
maxPrecision = 11
for i in range(1, maxPrecision):
    newData, newLength = splitEdges(i)
    newData = np.asarray(newData)
    print(i, newLength)
    newData = newData.reshape((newData.shape[0], 1))
    data = np.concatenate((data, newData), axis=1)

kdf = ks.DataFrame(data=data, columns=range(maxPrecision))
kdf.describe()

0 285186
1 570372
2 855558
3 1140744
4 1425930
5 1711116
6 1996302
7 2281488
8 2566674
9 2851860
10 3137046


Py4JJavaError: An error occurred while calling o3043.summary.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 3 in stage 50.0 failed 1 times, most recent failure: Lost task 3.0 in stage 50.0 (TID 55, localhost, executor driver): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/home/cristiano/anaconda3/lib/python3.7/site-packages/pyspark/python/lib/pyspark.zip/pyspark/worker.py", line 267, in main
    ("%d.%d" % sys.version_info[:2], version))
Exception: Python in worker has different version 2.7 than that in driver 3.7, PySpark cannot run with different minor versions.Please check environment variables PYSPARK_PYTHON and PYSPARK_DRIVER_PYTHON are correctly set.

	at org.apache.spark.api.python.BasePythonRunner$ReaderIterator.handlePythonException(PythonRunner.scala:456)
	at org.apache.spark.api.python.PythonRunner$$anon$1.read(PythonRunner.scala:592)
	at org.apache.spark.api.python.PythonRunner$$anon$1.read(PythonRunner.

In [11]:
data, newLength = splitEdges(maxDistance=200)
print(-1, newLength)
kdfDistance = pd.DataFrame(data)
kdfDistance.describe()

-1 296660


,0
count,285186.000000
mean,65.928855
std,45.726769
min,0.015059
25%,28.049684
50%,58.427600
75%,96.590991
max,199.998005


In [12]:
def formatFloat(value):
    numDecimaPlaces = 4
    return " & " + "{:.{nDigits}f}".format(value, nDigits=numDecimaPlaces)

#stringKdf = kdf.describe().to_string(float_format=formatFloat).replace("\n", " \\\\\n")
#print(stringKdf)

stringKdf = kdfDistance.describe().to_string(float_format=formatFloat).replace("\n", " \\\\\n")
print(stringKdf)

                   0 \\
count  & 285186.0000 \\
mean       & 65.9289 \\
std        & 45.7268 \\
min         & 0.0151 \\
25%        & 28.0497 \\
50%        & 58.4276 \\
75%        & 96.5910 \\
max       & 199.9980
